** Automated Box Label Creator ** 

This script takes as information a folder with several PDF invoices.

It generates Box Labels corresponding to each volume per invoice, fills all the required fields in the box label, namely the corresponding order note, invoice number and package sequence per invoice.

It also generates the corresponding packing list for that pallet, which includes all relevant information.

############################################################################################################

In [ ]:
import os
import re
import fitz  # PyMuPDF
from fpdf import FPDF

def extract_invoice_number(filename):
    match = re.search(r'\d{4}(?=[^\d]*$)', filename)
    if match:
        return int(match.group(0))  # Convert to integer for numeric sorting
    return None

def extract_order_and_volumes(pdf_content):
    order_number = None
    volumes = None
    
    lines = pdf_content.splitlines()
    
    for line in lines:
        order_match = re.search(r'\b990\d{7}\b', line)
        if order_match:
            order_number = order_match.group(0)
            break
    
    for line in lines:
        volumes_match = re.search(r'volumes:\s*(\d+)', line, re.IGNORECASE)
        if volumes_match:
            volumes = int(volumes_match.group(1))
            break
    
    return order_number, volumes

def extract_invoice_details(invoices_dir, output_filename="invoice_details.txt"):
    with open(output_filename, 'w') as f_out:
        pdf_files = []
        for filename in os.listdir(invoices_dir):
            if filename.endswith('.pdf'):
                pdf_files.append(filename)

        pdf_files.sort(key=lambda x: extract_invoice_number(x))
        
        invoice_count = 0  # Initialize invoice count

        for filename in pdf_files:
            pdf_path = os.path.join(invoices_dir, filename)
            invoice_number = extract_invoice_number(filename)
            if invoice_number:
                try:
                    pdf_document = fitz.open(pdf_path)
                    pdf_text = ""
                    for page_num in range(len(pdf_document)):
                        pdf_text += pdf_document[page_num].get_text()
                    pdf_document.close()
                    invoice_count += 1  # Increment invoice count
                    order_number, num_volumes = extract_order_and_volumes(pdf_text)
                    
                    for volume in range(1, num_volumes + 1):
                        f_out.write(f"De: Experience Source\n")
                        f_out.write(f"Para: FNAC\n")
                        f_out.write(f"Pedido: {order_number}\n")
                        f_out.write(f"Tipo: 2004/0902\n")
                        f_out.write(f"Nossa Fatura: {invoice_number}\n")
                        f_out.write(f"Ref: {invoice_count}\n")  # Write current invoice count
                        f_out.write(f"Volume: {volume}/{num_volumes}\n")
                        f_out.write("\n")
                
                except Exception as e:
                    print(f"Error processing {pdf_path}: {str(e)}")

def generate_packing_list(invoices_dir, output_filename="packing_list.pdf"):
    from reportlab.lib.pagesizes import A4
    from reportlab.lib.units import mm
    from reportlab.pdfgen import canvas

    PAGE_WIDTH, PAGE_HEIGHT = A4
    PAGE_MARGIN = 10 * mm

    c = canvas.Canvas(output_filename, pagesize=A4)

    c.setStrokeColorRGB(0, 0, 0)
    c.setLineWidth(2)
    c.rect(PAGE_MARGIN, PAGE_MARGIN, PAGE_WIDTH - 2 * PAGE_MARGIN, PAGE_HEIGHT - 2 * PAGE_MARGIN)

    c.setFont("Helvetica-Bold", 12)
    current_y = PAGE_HEIGHT - PAGE_MARGIN - 30 * mm
    c.drawCentredString(PAGE_WIDTH / 2, current_y, "PACKING LIST - EXPERIENCE SOURCE LDA (FORN. 210407)")
    current_y -= 20 * mm

    c.drawCentredString(PAGE_WIDTH / 2, current_y, "CLIENTE: FNAC PORTUGAL - ARMAZÉM ALVERCA")
    current_y -= 20 * mm

    c.drawCentredString(PAGE_WIDTH / 2, current_y, "Morada de entrega : ALVERCA PARK, C 7 - ESTRADA NACIONAL 10")
    current_y -= 20 * mm

    c.setFont("Helvetica-Bold", 10)
    c.drawString(PAGE_MARGIN + 20 * mm, current_y, "Ref.")
    c.drawString(PAGE_MARGIN + 30 * mm, current_y, "Volumes")
    c.drawString(PAGE_MARGIN + 80 * mm, current_y, "Nota de Encomenda")
    c.drawString(PAGE_MARGIN + 160 * mm, current_y, "Fatura")
    current_y -= 15

    c.setFont("Helvetica", 10)
    row_height = 15

    pdf_files = []
    for filename in os.listdir(invoices_dir):
        if filename.endswith('.pdf'):
            pdf_files.append(filename)

    pdf_files.sort(key=lambda x: extract_invoice_number(x))

    invoice_index = 1

    for filename in pdf_files:
        pdf_path = os.path.join(invoices_dir, filename)
        invoice_number = extract_invoice_number(filename)
        if invoice_number:
            try:
                pdf_document = fitz.open(pdf_path)
                pdf_text = ""
                for page_num in range(len(pdf_document)):
                    pdf_text += pdf_document[page_num].get_text()
                pdf_document.close()
                
                order_number, num_volumes = extract_order_and_volumes(pdf_text)
                
                c.drawString(PAGE_MARGIN + 20 * mm, current_y, str(invoice_index))
                c.drawString(PAGE_MARGIN + 30 * mm, current_y, str(num_volumes))
                c.drawString(PAGE_MARGIN + 80 * mm, current_y, order_number)
                c.drawString(PAGE_MARGIN + 160 * mm, current_y, str(invoice_number))
                current_y -= row_height

                invoice_index += 1
            
            except Exception as e:
                print(f"Error processing {pdf_path}: {str(e)}")

            if current_y <= PAGE_MARGIN + 15:
                c.showPage()
                current_y = PAGE_HEIGHT - PAGE_MARGIN - 30 * mm

    c.save()

def generate_box_labels(input_filename='invoice_details.txt', output_filename='box_labels.pdf'):
    class PDF(FPDF):
        def header(self):
            pass

        def footer(self):
            pass

    pdf = PDF()
    pdf.set_auto_page_break(auto=False)  # Disable auto page break to control it manually
    pdf.add_page()

    with open(input_filename, 'r') as f_in:
        box_labels = f_in.readlines()

    line_count = 0
    line_height = 6  # Adjust line height as needed, smaller value reduces spacing
    lines_per_page = 56  # Number of lines after which to add a new page
    current_x, current_y = 10, 10
    max_x = 105  # Middle of A4 width (210 mm / 2)
    pdf.set_font("Arial", size=15)

    for line in box_labels:
        if line.strip():
            if line_count % lines_per_page == 0 and line_count != 0:
                pdf.add_page()
                current_x, current_y = 10, 10
            
            pdf.set_xy(current_x, current_y)
            pdf.multi_cell(0, line_height, line.strip(), align='L')
            current_y += line_height
            line_count += 1

            if line_count % 7 == 0:
                current_y += 10  # Add a blank line between each set of 7 lines (box label)
                if current_x == 10:
                    current_x = max_x  # Switch to the right column
                    current_y -= (7 * line_height + 10)  # Reset Y to align with the left column
                else:
                    current_x = 10  # Switch back to the left column
                    current_y += (10)  # Move to next label position in left column

    pdf.output(output_filename)

if __name__ == "__main__":
    invoices_dir = 'invoices'
    invoice_details_output_file = 'invoice_details.txt'
    packing_list_output_file = 'packing_list.pdf'
    box_labels_output_file = 'box_labels.pdf'

    extract_invoice_details(invoices_dir, invoice_details_output_file)
    generate_packing_list(invoices_dir, packing_list_output_file)
    generate_box_labels(input_filename=invoice_details_output_file, output_filename=box_labels_output_file)

    with open(invoice_details_output_file, 'r') as f:
        box_labels_count = len(f.readlines())
    
    print(f"Packing list created successfully")
    print(f"Box label PDF created successfully")
    print(f"Generated {box_labels_count // 8} box labels")


############################################################################################################

Some examples of the generated documents :

<img src="BL1.png" alt="BL1" width="300"/>

<img src="PL1.png" alt="PL1" width="300"/>